# Synthetic drifter trajectories exponentially autocorrelated velocities

We generate synthetic exponentially autocorrelated velocity timeseries.
Displacement and acceleration timeseries are derived.

We verify pure exponentially decorrelated timeseries are not differentiable and lead to white noise acceleration.
A modification leading to differentiable solutions is used and consists in (multi-layer) embeded Ornstein–Uhlenbeck (OU) processes [Sawford 1991, Viggiano et al. 2020].
All processes are assumed Gaussian and could be extended to more general distributions [Viggiano et al. 2020].

References:

- B. Viggiano, J. Friedrich, R. Volk, M. Bourgoin, R. B. Cal, and L. Chevillard. Modelling lagrangian velocity and acceleration in turbulent flows as infinitely differentiable stochastic processes. Journal of Fluid Mechanics, 900, 2020.
- B. Sawford. Reynolds number effects in lagrangian stochastic models of turbulent dispersion. Physics of Fluids A: Fluid Dynamics, 3(6):1577–1586, 1991.


In [8]:
import os
import xarray as xr
import numpy as np


import matplotlib.pyplot as plt
from cycler import cycler

from sstats import signals as sg
from sstats import sigp as sigp
from sstats import tseries as ts
from sstats import get_cmap_colors

from GDP_lib import root_dir

---

## Exponentially autocorrelated low-frequency velocity signal :  n=1


In [2]:
u_low = sg.low_frequency_signal()

# note we also could consider / add a high frequency component
# u_high = sg.high_frequency_signal()

- U = signal amplitude (standard deviation)
- $T$ = signal decorrelation time scale (exponential autocorrelation)

In [3]:
T, U = u_low.p["T"], u_low.p["U"]
# sigma = u_high.p['sigma']
omega, tau = u_low.omega, u_low.tau

# print analytical spectrum
u_low.spectrum

2*T*U**2/(T**2*omega**2 + 1)

Such spectrum leads to a flat acceleration spectrum at high frequencies which cannot be integrated and suggests acceleration is white noise.

In [5]:
# decorrelation timescales
T = [5,10,20,40]
t = (1000, 1 / 24) # timeline: 1000 days with hourly sampling
N = 10 # number of random draws

#### Illustration

In [ ]:
# generate velocity first
da = u_low.generate_tseries(
    params=dict(T=T),
    draws=N,
    time=t,
    seed=0,
)

# derive position and acceleration
ds = da.rename("u").assign_attrs(units="m/s").to_dataset().persist()
ds["time"] = ds["time"].assign_attrs(units="days")
ds["x"] = ds["u"].cumulative_integrate("time").assign_attrs(units="m") *86400 # rescale in m
ds["a"] = ds["u"].differentiate("time").assign_attrs(units="m/s^2") /86400  # rescale in m/s
ds = ds.persist()

# compute autocorrelations
xx = sigp.correlate(ds.x, ds.x).rename("xx").persist().assign_attrs(long_name=r'$C_x(\tau)$', units="")
uu = sigp.correlate(ds.u, ds.u).rename("uu").persist().assign_attrs(long_name=r'$C_u(\tau)$', units="")
aa = sigp.correlate(ds.a, ds.a).rename("aa").persist().assign_attrs(long_name=r'$C_a(\tau)$', units="")
dsc = xr.merge([xx, uu, aa])

# compute spectra
Ex, omega = sigp.spectrum_welch(ds.x, T=100)
Eu, omega = sigp.spectrum_welch(ds.u, T=100)
Ea, omega = sigp.spectrum_welch(ds.a, T=100)
dsE = xr.merge([Ex.assign_attrs(units=r"$m^2/cpd$"), Eu.assign_attrs(units=r"$m^2/s^2/cpd$"), Ea.assign_attrs(units=r"$m^2/s^4/cpd$")])
dsE = dsE.rename({i : 'E'+i for i in list(dsE.keys())})

ds_exp = xr.merge([ds, dsc, dsE])

ds_exp.T.attrs = {'long_name' : 'Correlation time T', 'units':'days'}
ds_exp.lags.attrs = {'long_name':r'$\tau$', 'units':'days'}
ds_exp.freq_time.attrs = { 'units':'cpd'}

ds_exp.attrs = {'description':'synthetic trajectories simulated via the embedded OU process of Viggiano et al. 2020 (equations (2.14-2.18))', 'n_layers':1, "sigma_u":u_std}

In [ ]:
ds_exp

In [9]:
ds_exp0 = ds_exp.chunk({'T':4,'time' : 5000, 'draw':-1, 'lags':5000})

In [10]:
ds_exp0

<xarray.Dataset>
Dimensions:    (T: 4, time: 24000, draw: 10, lags: 24000, freq_time: 2400)
Coordinates:
  * T          (T) int64 5 10 20 40
  * time       (time) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9
  * lags       (lags) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * freq_time  (freq_time) float64 -12.0 -11.99 -11.98 ... 11.97 11.98 11.99
Data variables:
    u          (T, time, draw) float64 dask.array<chunksize=(4, 5000, 10), meta=np.ndarray>
    x          (T, time, draw) float64 dask.array<chunksize=(4, 5000, 10), meta=np.ndarray>
    a          (T, time, draw) float64 dask.array<chunksize=(4, 5000, 10), meta=np.ndarray>
    xx         (T, draw, lags) float64 dask.array<chunksize=(4, 10, 5000), meta=np.ndarray>
    uu         (T, draw, lags) float64 dask.array<chunksize=(4, 10, 5000), meta=np.ndarray>
    aa         (T, draw, lags) float64 dask.array<chunksize=(4, 10, 5000), meta=np.ndarray>
    Ex         (T, draw, freq_time) float64 dask.array<chunksize=(4, 10, 2400), meta=np.ndarray>
    Eu         (T, draw, freq_time) float64 dask.array<chunksize=(4, 10, 2400), meta=np.ndarray>
    Ea         (T, draw, freq_time) float64 dask.array<chunksize=(4, 10, 2400), meta=np.ndarray>

In [11]:
#root_dir ='/Users/mdemol/code/PhD/GDP/synthetics_trajectories'
path = os.path.join(root_dir, "synthetic_traj_exp.zarr")

ds_exp0.to_zarr(path, mode="w")

In [12]:
dsl = xr.open_zarr(path, decode_timedelta=False)
dsl

<xarray.Dataset>
Dimensions:    (T: 4, draw: 10, freq_time: 2400, time: 24000, lags: 24000)
Coordinates:
  * T          (T) int64 5 10 20 40
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9
  * freq_time  (freq_time) float64 -12.0 -11.99 -11.98 ... 11.97 11.98 11.99
  * lags       (lags) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * time       (time) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
Data variables:
    Ea         (T, draw, freq_time) float64 dask.array<chunksize=(4, 10, 2400), meta=np.ndarray>
    Eu         (T, draw, freq_time) float64 dask.array<chunksize=(4, 10, 2400), meta=np.ndarray>
    Ex         (T, draw, freq_time) float64 dask.array<chunksize=(4, 10, 2400), meta=np.ndarray>
    a          (T, time, draw) float64 dask.array<chunksize=(4, 5000, 10), meta=np.ndarray>
    aa         (T, draw, lags) float64 dask.array<chunksize=(4, 10, 5000), meta=np.ndarray>
    u          (T, time, draw) float64 dask.array<chunksize=(4, 5000, 10), meta=np.ndarray>
    uu         (T, draw, lags) float64 dask.array<chunksize=(4, 10, 5000), meta=np.ndarray>
    x          (T, time, draw) float64 dask.array<chunksize=(4, 5000, 10), meta=np.ndarray>
    xx         (T, draw, lags) float64 dask.array<chunksize=(4, 10, 5000), meta=np.ndarray>

In [14]:
xr.open_zarr?

Signature:
xr.open_zarr(
    store,
    group=None,
    synchronizer=None,
    chunks='auto',
    decode_cf=True,
    mask_and_scale=True,
    decode_times=True,
    concat_characters=True,
    decode_coords=True,
    drop_variables=None,
    consolidated=None,
    overwrite_encoded_chunks=False,
    chunk_store=None,
    storage_options=None,
    decode_timedelta=None,
    use_cftime=None,
    **kwargs,
)
Docstring:
Load and decode a dataset from a Zarr store.

The `store` object should be a valid store for a Zarr group. `store`
variables must contain dimension metadata encoded in the
`_ARRAY_DIMENSIONS` attribute or must have NCZarr format.

Parameters
----------
store : MutableMapping or str
    A MutableMapping where a Zarr Group has been stored or a path to a
    directory in file system where a Zarr DirectoryStore has been stored.
synchronizer : object, optional
    Array synchronizer provided to zarr
group : str, optional
    Group path. (a.k.a. `path` in zarr terminology.)
chun

---

## Embedded OU processes n-layers: Viggiano et al. 2020

We generate timeseries corresponding to their equations (2.14-2.18).

In [3]:
# decorrelation timescales
T = [5,10,20,40]
t = (1e3, 1 / 24) # timeline: 10 000 days with hourly sampling
N = 10 # number of random draws

In [9]:
tau_eta = [1/48,1/24, 1/12,1,2]
#tau_eta = 1/24 # short timescale
n_layers = 5 # number of layers
u_std = 1 
da = u_std * ts.spectral_viggiano(t, T, tau_eta, n_layers, draws=N).compute()
print(' ok0')
# derive position and acceleration
ds = da.rename("u").assign_attrs(units="m/s").to_dataset().persist()
ds["time"] = ds["time"].assign_attrs(units="days")
ds["x"] = (ds["u"].cumulative_integrate("time")*86400).assign_attrs(units="m")
ds["a"] = (ds["u"].differentiate("time")/86400).assign_attrs(units=r"$m/s^2$")  # rescale in m/s
ds = ds.chunk({'T':-1, 'time' :24000, 'draw':-1}).persist()
print(' ok1')


# compute autocorrelations
xx = sigp.correlate(ds.x, ds.x).rename("xx").persist().assign_attrs(long_name=r'$C_x(\tau)$', units="")
print(' ok2')
uu = sigp.correlate(ds.u, ds.u).rename("uu").persist().assign_attrs(long_name=r'$C_u(\tau)$', units="")
print(' ok3')
aa = sigp.correlate(ds.a, ds.a).rename("aa").persist().assign_attrs(long_name=r'$C_a(\tau)$', units="")
dsc = xr.merge([xx, uu, aa])
print(' ok4')


# compute spectra
Ex, omega = sigp.spectrum_welch(ds.x, T=100)
print(' ok5')
Eu, omega = sigp.spectrum_welch(ds.u, T=100)
print(' ok6')
Ea, omega = sigp.spectrum_welch(ds.a, T=100)
dsE = xr.merge([Ex.assign_attrs(long_name = 'Power density spectra of x', units=r"$m^2/cpd$"),
                Eu.assign_attrs(long_name = 'Power density spectra of u',units=r"$m^2/s^2/cpd$"), 
                Ea.assign_attrs(long_name = 'Power density spectra of a',units=r"$m^2/s^4/cpd$")])
dsE = dsE.rename({i : 'E'+i for i in list(dsE.keys())})
ds_ou = xr.merge([ds, dsc, dsE])
if 'tau_eta' in list(ds_ou.coords):
    ds_ou.tau_eta.attrs = {'long_name' : r'$\tau_{\eta}', 'units':'days'}
ds_ou.T.attrs = {'long_name' : 'Correlation time T', 'units':'days'}
ds_ou.lags.attrs = {'long_name':r'$\tau$', 'units':'days'}
ds_ou.freq_time.attrs = { 'units':'cpd'} 
ds_ou.attrs = {'description':'synthetic trajectories simulated via the embedded OU process of Viggiano et al. 2020 (equations (2.14-2.18))', 'tau_eta_days':tau_eta, 'n_layers':n_layers, "sigma_u":u_std}
ds_ou

 ok0
 ok1
 ok2
 ok3
 ok4
 ok5
 ok6


<xarray.Dataset>
Dimensions:    (T: 4, tau_eta: 5, time: 24000, draw: 10, lags: 24000,
                freq_time: 2400)
Coordinates:
  * T          (T) int64 5 10 20 40
  * tau_eta    (tau_eta) float64 0.02083 0.04167 0.08333 1.0 2.0
  * time       (time) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9
  * lags       (lags) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * freq_time  (freq_time) float64 -12.0 -11.99 -11.98 ... 11.97 11.98 11.99
Data variables:
    u          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 24000, 10), meta=np.ndarray>
    x          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 24000, 10), meta=np.ndarray>
    a          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 24000, 10), meta=np.ndarray>
    xx         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 24000), meta=np.ndarray>
    uu         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 24000), meta=np.ndarray>
    aa         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 24000), meta=np.ndarray>
    Ex         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
    Eu         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
    Ea         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
Attributes:
    description:   synthetic trajectories simulated via the embedded OU proce...
    tau_eta_days:  [0.020833333333333332, 0.041666666666666664, 0.08333333333...
    n_layers:      5
    sigma_u:       1

In [11]:
if 'tau_eta' in list(ds_ou.coords):
    ds_ou.tau_eta.attrs = {'long_name' : r'$\tau_{\eta}$', 'units':'days'}

In [12]:
ds_ou

<xarray.Dataset>
Dimensions:    (T: 4, tau_eta: 5, time: 24000, draw: 10, lags: 24000,
                freq_time: 2400)
Coordinates:
  * T          (T) int64 5 10 20 40
  * tau_eta    (tau_eta) float64 0.02083 0.04167 0.08333 1.0 2.0
  * time       (time) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9
  * lags       (lags) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * freq_time  (freq_time) float64 -12.0 -11.99 -11.98 ... 11.97 11.98 11.99
Data variables:
    u          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 24000, 10), meta=np.ndarray>
    x          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 24000, 10), meta=np.ndarray>
    a          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 24000, 10), meta=np.ndarray>
    xx         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 24000), meta=np.ndarray>
    uu         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 24000), meta=np.ndarray>
    aa         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 24000), meta=np.ndarray>
    Ex         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
    Eu         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
    Ea         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
Attributes:
    description:   synthetic trajectories simulated via the embedded OU proce...
    tau_eta_days:  [0.020833333333333332, 0.041666666666666664, 0.08333333333...
    n_layers:      5
    sigma_u:       1

In [13]:

#root_dir ='/Users/mdemol/code/PhD/GDP/synthetics_trajectories'
path = os.path.join(root_dir, "synthetic_traj_ou_taueta.zarr")
#path = os.path.join(root_dir, "synthetic_traj_ou_tau.zarr")
ds_ou.chunk({'T':-1,'tau_eta':-1, 'time' : 5000, 'draw':-1, 'lags':5000}).to_zarr(path, mode="w")
#ds_ou.chunk({'T':-1, 'time' : 5000, 'draw':-1, 'lags':5000}).to_zarr(path, mode="a", append_dim = 'draw')

In [14]:
dsl =  xr.open_zarr(path, decode_timedelta=False)
#dsl['draw']=xr.DataArray(np.arange(100), dims=['draw'])

In [15]:
dsl

<xarray.Dataset>
Dimensions:    (T: 4, tau_eta: 5, draw: 10, freq_time: 2400, time: 24000,
                lags: 24000)
Coordinates:
  * T          (T) int64 5 10 20 40
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9
  * freq_time  (freq_time) float64 -12.0 -11.99 -11.98 ... 11.97 11.98 11.99
  * lags       (lags) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
  * tau_eta    (tau_eta) float64 0.02083 0.04167 0.08333 1.0 2.0
  * time       (time) float64 0.0 0.04167 0.08333 0.125 ... 999.9 999.9 1e+03
Data variables:
    Ea         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
    Eu         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
    Ex         (T, tau_eta, draw, freq_time) float64 dask.array<chunksize=(4, 5, 10, 2400), meta=np.ndarray>
    a          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 5000, 10), meta=np.ndarray>
    aa         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 5000), meta=np.ndarray>
    u          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 5000, 10), meta=np.ndarray>
    uu         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 5000), meta=np.ndarray>
    x          (T, tau_eta, time, draw) float64 dask.array<chunksize=(4, 5, 5000, 10), meta=np.ndarray>
    xx         (T, tau_eta, draw, lags) float64 dask.array<chunksize=(4, 5, 10, 5000), meta=np.ndarray>
Attributes:
    description:   synthetic trajectories simulated via the embedded OU proce...
    n_layers:      5
    sigma_u:       1
    tau_eta_days:  [0.020833333333333332, 0.041666666666666664, 0.08333333333...

---

## white noise

Generating white noise to add to position or velocity

In [17]:
noise_std = 1.

# first method: common noise realization 
from numpy.random import randn
ds["noise_c"] = ("time", randn(ds.time.size)*noise_std)

# second method: independent noise realizations
# scale represents the noise 
ds["noise_i"] = ts.normal(t, draws=N)*noise_std